# Script to get the jacobian matrix which under the .ipynb format

In [2]:
import sympy as sym
import numpy as np

In [3]:
x, y, z, w, h, l, v, a, yaw, ry_rate = sym.symbols("x y z w h l v a yaw ry_rate")
next_x, next_y, next_z, next_w, next_h, next_l, next_v, next_a, next_yaw, next_ry_rate = sym.symbols("next_x next_y next_z next_w next_h next_l next_v next_a next_yaw next_ry_rate")
dt = sym.symbols("dt")

In [4]:
next_z, next_w, next_h, next_l, next_a, next_ry_rate = z, w, h, l, a, ry_rate
next_v = v + a * dt
next_yaw = yaw + ry_rate * dt

In [5]:
vt, yawt, ita, t = sym.symbols("vt yawt ita t")
"""
vt = v + a * (ita - (t - 1) * dt)
yawt = yaw + ry_rate * (ita - (t - 1) * dt)
next_x = x + sym.integrate(vt * sym.cos(yawt), (ita, (t - 1) * dt, t * dt))
next_y = y + sym.integrate(vt * sym.sin(yawt), (ita, (t - 1) * dt, t * dt))
"""
# warp raw transition time from [(t-1)*dt, t*dt] to [0, dt], for quick solution
vt = v + a * (ita)
yawt = yaw + ry_rate * (ita)
next_x = x + sym.integrate(vt * sym.cos(yawt), (ita, 0, dt))
next_y = y + sym.integrate(vt * sym.sin(yawt), (ita, 0, dt))

# Analytical solutions for displacements can also be obtained

In [6]:
next_x

x + Piecewise((a*dt*sin(dt*ry_rate + yaw)/ry_rate - a*cos(yaw)/ry_rate**2 + a*cos(dt*ry_rate + yaw)/ry_rate**2 - v*sin(yaw)/ry_rate + v*sin(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), ((a*dt**2/2 + dt*v)*cos(yaw), True))

In [7]:
next_y

y + Piecewise((-a*dt*cos(dt*ry_rate + yaw)/ry_rate - a*sin(yaw)/ry_rate**2 + a*sin(dt*ry_rate + yaw)/ry_rate**2 + v*cos(yaw)/ry_rate - v*cos(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), ((a*dt**2/2 + dt*v)*sin(yaw), True))

In [8]:
funcs = sym.Matrix([next_x, next_y, next_z, next_w, next_l, next_h, next_v, next_a, next_yaw, next_ry_rate])
args = sym.Matrix([x, y, z, w, l, h, v, a, yaw, ry_rate])
res = funcs.jacobian(args)

In [9]:
res

Matrix([
[1, 0, 0, 0, 0, 0, Piecewise((-sin(yaw)/ry_rate + sin(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (dt*cos(yaw), True)),  Piecewise((dt*sin(dt*ry_rate + yaw)/ry_rate - cos(yaw)/ry_rate**2 + cos(dt*ry_rate + yaw)/ry_rate**2, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (dt**2*cos(yaw)/2, True)), Piecewise((a*dt*cos(dt*ry_rate + yaw)/ry_rate + a*sin(yaw)/ry_rate**2 - a*sin(dt*ry_rate + yaw)/ry_rate**2 - v*cos(yaw)/ry_rate + v*cos(dt*ry_rate + yaw)/ry_rate, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (-(a*dt**2/2 + dt*v)*sin(yaw), True)), Piecewise((a*dt**2*cos(dt*ry_rate + yaw)/ry_rate - 2*a*dt*sin(dt*ry_rate + yaw)/ry_rate**2 + 2*a*cos(yaw)/ry_rate**3 - 2*a*cos(dt*ry_rate + yaw)/ry_rate**3 + dt*v*cos(dt*ry_rate + yaw)/ry_rate + v*sin(yaw)/ry_rate**2 - v*sin(dt*ry_rate + yaw)/ry_rate**2, (ry_rate > -oo) & (ry_rate < oo) & Ne(ry_rate, 0)), (0, True))],
[0, 1, 0, 0, 0, 0,  Piecewise((cos(yaw)/ry_rate - cos(dt*ry_rate + yaw)/ry_rate,

In [10]:
mea_funcs = sym.Matrix([x, y, z, w, l, h, v * sym.cos(yaw), v * sym.sin(yaw), yaw])
mea_res = mea_funcs.jacobian(args)

In [11]:
mea_res

Matrix([
[1, 0, 0, 0, 0, 0,        0, 0,           0, 0],
[0, 1, 0, 0, 0, 0,        0, 0,           0, 0],
[0, 0, 1, 0, 0, 0,        0, 0,           0, 0],
[0, 0, 0, 1, 0, 0,        0, 0,           0, 0],
[0, 0, 0, 0, 1, 0,        0, 0,           0, 0],
[0, 0, 0, 0, 0, 1,        0, 0,           0, 0],
[0, 0, 0, 0, 0, 0, cos(yaw), 0, -v*sin(yaw), 0],
[0, 0, 0, 0, 0, 0, sin(yaw), 0,  v*cos(yaw), 0],
[0, 0, 0, 0, 0, 0,        0, 0,           1, 0]])